# ACORN-SAT using Python and Jupyter Notebook

## Let's load some data...

In [ ]:
import io
import requests
import pandas as pd

response = requests.get('http://www.bom.gov.au/climate/change/acorn/sat/data/acorn.sat.minT.086071.daily.txt')
file_object = io.StringIO(response.content.decode('utf-8'))

tmin_melb = pd.read_csv(file_object)

tmin_melb.head()

In [ ]:
import pandas as pd

url = "http://www.bom.gov.au/climate/change/acorn/sat/data/acorn.sat.minT.086071.daily.txt"
tmin_melb = pd.read_csv(url)

tmin_melb.head()

In [ ]:
import pandas as pd

tmin_melb = pd.read_csv('86071_tmin.csv', delimiter='\s+', skiprows=1,)

tmin_melb.head()

In [ ]:
import pandas as pd

tmin_melb = pd.read_csv('86071_tmin.csv', delimiter='\s+', skiprows=1, names=['date', 'tmin'], na_values='99999.9',)

tmin_melb.head()

In [ ]:
import pandas as pd
import datetime

# Function to convert YYYYMMDD to Python datetime
def make_date(n):
    return datetime.datetime.strptime(n, '%Y%m%d')

url = "http://www.bom.gov.au/climate/change/acorn/sat/data/acorn.sat.minT.086071.daily.txt"
tmin_melb = pd.read_csv('86071_tmin.csv', delimiter='\s+', skiprows=1, names=['date', 'tmin'], na_values='99999.9',
                       index_col=0, converters={'date':make_date})

tmin_melb.head(5)

In [ ]:
# http://www.bom.gov.au/climate/change/hqsites/data/temp/tmin.086338.daily.csv
import pandas as pd
import datetime

# Function to convert YYYYMMDD to Python datetime
def make_date(n):
    return datetime.datetime.strptime(n, '%Y-%m-%d')

url = "http://www.bom.gov.au/climate/change/hqsites/data/temp/tmin.086338.daily.csv"

tmin_melb = pd.read_csv(url, skiprows=2,  index_col=0, usecols=[0,1], names=['date', 'tmin'],
                       converters={'date': make_date})
tmin_melb.head(5)

In [ ]:
import pandas as pd
import datetime

# Function to convert YYYYMMDD to Python datetime
def make_date(n):
    return pd.Timestamp.strptime(n, '%Y%m%d')

# Get tmin for Melbourne
tmin_melb = pd.read_csv('86071_tmin.csv', delimiter='\s+', skiprows=1, names=['date', 'tmin'], na_values='99999.9',
                       index_col=0, converters={'date':make_date})

# Get tmax for Melbourne
url = "http://www.bom.gov.au/climate/change/acorn/sat/data/acorn.sat.maxT.086071.daily.txt"
tmax_melb = pd.read_csv('86071_tmax.csv', delimiter='\s+', skiprows=1, names=['date', 'tmax'], na_values='99999.9',
                       index_col=0, converters={'date':make_date})

tmax_melb.head(5)

In [ ]:
import pandas as pd

tall_melb = tmin_melb.join(tmax_melb, how='outer')
tall_melb.head(10)

In [ ]:
tall_melb['tmean'] = (tall_melb['tmin'] + tall_melb['tmax']) / 2
tall_melb.head(10)

In [ ]:
tall_melb['tave'] = (tall_melb['tmin'].shift(1) + tall_melb['tmax']) / 2
tall_melb.head(10)

In [ ]:
tall_melb.describe()

In [ ]:
tall_melb = tall_melb.drop('tave', axis=1)
tall_melb.head(10)

In [ ]:
%matplotlib inline

tall_melb.plot()

In [ ]:
tall_melb['1980-01-01':'1980-12-31'].plot()

In [ ]:
tall_melb['1980-01-01':'1980-12-31'].rolling(10).mean().plot()

In [ ]:
%matplotlib inline

tall_melb['1980-01-01':'1980-12-31'].rolling(10, win_type='triang').mean().plot() 
# boxcar triang blackman hamming bartlett parzen bohman blackmanharris nuttall barthann

In [ ]:
%matplotlib inline

# Can also use an offset - e.g. number of days...
tall_melb['1980-01-01':'1980-12-31'].rolling('10d').mean().plot()

In [ ]:
# Group by month...

mnth_melb = tall_melb.groupby(by=tall_melb.index.month)
mnth_melb.mean()

In [ ]:
# Find the monthly climatology for Melbourne...

clim = tall_melb['1961-01-01':'1991-01-01']
clim.groupby(by=clim.index.month).mean()

In [ ]:
tall_melb['tmax'] > 42

In [ ]:
# Find all days with max temp > 42

# tall_melb['tmax'] > 42
tall_melb[tall_melb['tmax'] > 42]

In [ ]:
# Look at correlation...

tall_melb['tmax'].corr(tall_melb['tmin'])

In [ ]:
tall_melb.corr()

In [ ]:
tall_melb.quantile(0.5)

In [ ]:
tall_melb.quantile([0.1, 0.5, 0.9])

In [ ]:
tall_melb.quantile([0.1, 0.5, 0.9], interpolation='lower') # linear’, ‘lower’, ‘higher’, ‘midpoint’, ‘nearest’

In [ ]:
import seaborn as sns

correlations = tall_melb.corr()
sns.heatmap(correlations)

In [ ]:
# Find the 10 hottest days in Melbourne

tall_melb.sort_values(by='tmax', ascending=False).head(10)

In [ ]:
pd.plotting.lag_plot(tall_melb['2010-01-01':'2011-01-01']['tmax'])

In [ ]:
pd.plotting.autocorrelation_plot(tall_melb['2010-01-01':'2011-01-01']['tmax'])

In [ ]:
resampled = tall_melb['2010-01-01':'2011-01-01'].tmax.resample('W').mean()
resampled.plot()

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('acorn.db')

stations = pd.read_sql_query("SELECT * FROM stations", conn, index_col='stn_num')
stations.head(10)

In [ ]:
stations[['lat', 'lon']].values

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature 

ax = plt.axes(projection=ccrs.PlateCarree()) 
ax.set_extent([100, 170, -40, -5])         
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature 

ax = plt.axes(projection=ccrs.PlateCarree()) 
ax.set_extent([100, 170, -40, -5])         
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.scatter(stations.lon.values,stations.lat.values,transform=ccrs.PlateCarree())
plt.show()
# plt.savefig('acornsat.svg')

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('acorn.db')

data = pd.read_sql_query("""
    SELECT stn_num, lsd, prcp, t_min, t_max 
    FROM observations
    WHERE lsd >= '1960-01-01'
     AND  lsd < 1991-01-01
    """, conn)
data.head(10)

In [ ]:
data.groupby(by=data.stn_num).mean()


In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('acorn.db')

data = pd.read_sql_query("""
    SELECT stn_num, lsd, prcp, t_min, t_max 
    FROM observations
    WHERE lsd == '1960-01-01'
    """, conn)
data.head(10)


In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('acorn.db')

data = pd.read_sql_query("""
    SELECT lat, lon, s.stn_num,  t_max 
    FROM observations o, stations s
    WHERE lsd == '2005-01-01'
    AND o.stn_num = s.stn_num
    """, conn)
data.head(10)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

y = data.lat.values 
x = data.lon.values 
t = data.t_max.values
plt.scatter(x, y)
# plt.scatter(data.lon.values,data.lat.values)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

y = data.lat.values 
x = data.lon.values 
t = data.t_max.values
plt.scatter(x, y, c=t)
plt.colorbar()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

y = data.lat.values 
x = data.lon.values 
t = data.t_max.values
plt.scatter(x, y, c=t, s=t*5, alpha=0.5, cmap='plasma')
plt.colorbar()

In [ ]:
import numpy as np
from scipy.interpolate import griddata

X, Y = np.meshgrid(np.linspace(110,155,100), np.linspace(-45,-10,100))

#perform the gridding
grid_temp = griddata((x,y), t, (X, Y))

# plt.clf()
plt.contourf(X,Y,grid_temp, cmap='plasma')
plt.colorbar()

In [ ]:
x = tall_melb['1961-01-01':'1991-01-01'].tmin.rolling(5).mean()
x.quantile(0.1)
# tall_melb['1961-01-01':'1991-01-01'].tmin.quantile(0.1)

In [ ]:
x = tall_melb['1961-01-01':'1991-01-01'].tmin.rolling(5).mean()
q10 = x.quantile(0.1)
print(q10)
pop = tall_melb['1999-01-01':'2000-01-01']
total = pop.tmin.count()
sub = pop.tmin[pop.tmin < q10].count()
(sub / total) * 100

In [ ]:
import math

# a = [1, 1, 1, 1, 1, 1]

a = [1, 1, 1, 2, 3, 5, 7, 7, 11, 13, 13, 13]

b = [min(1, abs(x-y)) for x, y in zip(a, a[1:])] 


p = (b + [1]).index(1)
print(p)
a[:p] = [math.nan] * p
try:
    p = (b[::-1]).index(1)
    a[-p:] = [math.nan] * p
except:
    pass

a


In [ ]:
from mpl_toolkits import mplot3d

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# fig = plt.figure()
ax = plt.axes(projection="3d")

plt.show()

In [ ]:
# fig = plt.figure()
ax = plt.axes(projection="3d")

z_line = np.linspace(0, 15, 1000)
x_line = np.cos(z_line)
y_line = np.sin(z_line)
ax.plot3D(x_line, y_line, z_line, 'gray')

z_points = 15 * np.random.random(100)
x_points = np.cos(z_points) + 0.1 * np.random.randn(100)
y_points = np.sin(z_points) + 0.1 * np.random.randn(100)
ax.scatter3D(x_points, y_points, z_points, c=z_points, cmap='hsv');

plt.show()
